In [16]:
# ==========================================
# === CÉLULA 1: Instalações ===
# ==========================================
!pip install --quiet --upgrade langchain langchain-openai langchain-community faiss-cpu PyPDF2 tiktoken "unstructured[pdf]" pdfminer.six pdf2image pillow streamlit "aiofiles<24.0" pytesseract pyngrok
!apt-get update -qq
!apt-get install -qq poppler-utils tesseract-ocr tesseract-ocr-por

print("Instalações concluídas.")

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Instalações concluídas.


In [17]:
# ==========================================
# === CÉLULA 2: Imports Essenciais ===
# ==========================================
import os
import streamlit as st
from google.colab import userdata
from getpass import getpass
import numpy as np
import faiss
import tiktoken
import traceback # Para imprimir erros detalhados

# LangChain e relacionados
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document

# PDF e OCR
from PyPDF2 import PdfReader
from pdf2image import convert_from_path
import pytesseract

# Ngrok (para teste no Colab)
from pyngrok import ngrok

print("Imports concluídos.")

Imports concluídos.


In [27]:
!pip freeze > requirements.txt

In [18]:
# === CÉLULA 3: Configuração API Key, Repositório GitHub e Caminhos ===
try:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    chave_teste = os.environ["OPENAI_API_KEY"]
    print(f"✅ Chave OpenAI carregada com sucesso: {chave_teste[:5]}...{chave_teste[-4:]}")
except Exception as e:
    print(f"❌ Erro ao carregar chave OpenAI: {e}")

# --- Carregar GitHub PAT do Userdata ---
try:
    GITHUB_PAT = userdata.get('GITHUB_PAT')
    if not GITHUB_PAT:
        raise ValueError("Secret GITHUB_PAT está vazio ou não foi encontrado.")
    print("✅ GitHub PAT carregado do userdata.")
except Exception as e:
    print(f"❌ Erro ao carregar GITHUB_PAT do userdata: {e}")
    print("   Certifique-se de criar um secret chamado 'GITHUB_PAT' com seu Personal Access Token.")

    raise e # Parar execução se não conseguir carregar o PAT

# --- CONFIGURAÇÃO DO GITHUB ---
# Monta a URL com o PAT para autenticação
REPO_OWNER = "camila-f-romero"
REPO_NAME = "MBACDIA-IAGen"
REPO_URL_COM_PAT = f"https://{GITHUB_PAT}@github.com/{REPO_OWNER}/{REPO_NAME}.git"

NOME_REPO_LOCAL = REPO_NAME
NOME_PASTA_PDFS_NO_REPO = "arquivos-projeto"

# --- Clonar/Atualizar Repositório ---
DIRETORIO_ATUAL = os.getcwd()
print(f"Diretório de trabalho atual: {DIRETORIO_ATUAL}")
CAMINHO_REPO_LOCAL_ABS = os.path.join(DIRETORIO_ATUAL, NOME_REPO_LOCAL)
print(f"Caminho absoluto esperado para o repositório local: {CAMINHO_REPO_LOCAL_ABS}")

if not os.path.exists(CAMINHO_REPO_LOCAL_ABS):
    print(f"Clonando repositório (usando PAT) para '{CAMINHO_REPO_LOCAL_ABS}'...")
    # Usa a URL com o PAT. Não precisa mais de GIT_TERMINAL_PROMPT=0 ou -c credential.helper=
    ret_code = os.system(f"git clone {REPO_URL_COM_PAT} '{CAMINHO_REPO_LOCAL_ABS}'")
    if ret_code == 0:
        print("✅ Repositório clonado com sucesso.")
    else:
        print(f"❌ Falha ao clonar repositório (Código de retorno: {ret_code}). Verifique o PAT e a URL.")
        # Limpa o PAT da memória por segurança
        GITHUB_PAT = None
        raise RuntimeError("Falha no Git Clone com PAT")
else:
    print(f"Repositório '{NOME_REPO_LOCAL}' já existe em '{CAMINHO_REPO_LOCAL_ABS}'.")
    print("Verificando atualizações no repositório (usando PAT)...")

    ret_code = os.system(f"cd '{CAMINHO_REPO_LOCAL_ABS}' && git pull")
    if ret_code == 0:
         print("✅ Repositório atualizado com sucesso.")
    else:
         print(f"⚠️ Falha ao atualizar repositório com 'git pull' (Código: {ret_code}).")

# Limpa o PAT da memória após o uso inicial por segurança
GITHUB_PAT = None

# --- Define o Caminho para a Pasta de PDFs ---
CAMINHO_PASTA_PDFS = os.path.join(CAMINHO_REPO_LOCAL_ABS, NOME_PASTA_PDFS_NO_REPO)
print(f"Caminho completo esperado para a pasta de PDFs: {CAMINHO_PASTA_PDFS}")

# --- Verificação Final ---
print(f"\nVerificando novamente a existência de '{CAMINHO_REPO_LOCAL_ABS}'...")
if os.path.exists(CAMINHO_REPO_LOCAL_ABS):
    print(f"✅ Diretório do repositório '{NOME_REPO_LOCAL}' encontrado.")
    try:
        print(f"   Conteúdo encontrado na raiz do repositório clonado:", os.listdir(CAMINHO_REPO_LOCAL_ABS))
        print(f"\nVerificando a existência da pasta de PDFs '{CAMINHO_PASTA_PDFS}'...")
        if os.path.exists(CAMINHO_PASTA_PDFS):
            print(f"✅ Pasta de PDFs encontrada em: '{CAMINHO_PASTA_PDFS}'")
        else:
            print(f"❌ ATENÇÃO: A pasta de PDFs '{NOME_PASTA_PDFS_NO_REPO}' NÃO foi encontrada dentro de '{CAMINHO_REPO_LOCAL_ABS}'.")
            print(f"   Verifique se o nome '{NOME_PASTA_PDFS_NO_REPO}' está correto e se a pasta existe no repositório GitHub.")
    except Exception as list_err:
        print(f"❌ Erro ao listar conteúdo de '{CAMINHO_REPO_LOCAL_ABS}': {list_err}")
else:
    print(f"❌ Erro Crítico: O diretório do repositório '{NOME_REPO_LOCAL}' não foi encontrado após a tentativa de clonagem/atualização.")

✅ Chave OpenAI carregada com sucesso: sk-pr...kRwA
✅ GitHub PAT carregado do userdata.
Diretório de trabalho atual: /content
Caminho absoluto esperado para o repositório local: /content/MBACDIA-IAGen
Repositório 'MBACDIA-IAGen' já existe em '/content/MBACDIA-IAGen'.
Verificando atualizações no repositório (usando PAT)...
✅ Repositório atualizado com sucesso.
Caminho completo esperado para a pasta de PDFs: /content/MBACDIA-IAGen/arquivos-projeto

Verificando novamente a existência de '/content/MBACDIA-IAGen'...
✅ Diretório do repositório 'MBACDIA-IAGen' encontrado.
   Conteúdo encontrado na raiz do repositório clonado: ['.git', 'faiss_index_anpd_acts', 'arquivos-projeto']

Verificando a existência da pasta de PDFs '/content/MBACDIA-IAGen/arquivos-projeto'...
✅ Pasta de PDFs encontrada em: '/content/MBACDIA-IAGen/arquivos-projeto'


In [19]:
# === CÉLULA 4: Funções Auxiliares (Extração de Texto PDF/OCR) ===

# Função básica de extração com PyPDF2
def extrair_texto_pypdf2(pdf_path):
    texto = ""
    try:
        with open(pdf_path, "rb") as f:
            leitor = PdfReader(f)
            if leitor.is_encrypted:
                try:
                    leitor.decrypt("") # Tenta descriptografar com senha vazia
                except Exception as decrypt_err:
                    print(f"  -> Aviso: PDF criptografado e falha ao descriptografar '{os.path.basename(pdf_path)}': {decrypt_err}")
                    # return "" # Ou pode tentar extrair mesmo assim se a criptografia for leve

            for pagina in leitor.pages:
                try:
                    texto_pagina = pagina.extract_text()
                    if texto_pagina:
                        texto += texto_pagina + "\n" # Adicionar nova linha entre páginas
                except Exception as page_err:
                     print(f"  -> Erro ao extrair texto da página em '{os.path.basename(pdf_path)}': {page_err}")

    except Exception as e:
        print(f"  -> Erro PyPDF2 em '{os.path.basename(pdf_path)}': {e}")
    return texto.strip()

# Função de extração com OCR (Tesseract via pdf2image)
def extrair_texto_ocr(pdf_path):
    texto_total = ""
    print(f"  -> Tentando OCR em: {os.path.basename(pdf_path)}")
    try:
        imagens = convert_from_path(
            pdf_path,
            dpi=300, # Boa resolução para OCR
            poppler_path="/usr/bin",
            thread_count=2 # Ajuste se necessário
        )
        if not imagens:
             print(f"  -> Aviso: pdf2image não retornou imagens para OCR em '{os.path.basename(pdf_path)}'.")
             return ""

        for i, img in enumerate(imagens):
            try:
                texto_pagina = pytesseract.image_to_string(img, lang='por', config='--psm 6') # Tenta detectar layout
                texto_total += f"\n--- Página OCR {i+1} ---\n{texto_pagina}"
            except Exception as ocr_err:
                 print(f"    -> Erro OCR na página {i+1}: {ocr_err}")
        print(f"  -> OCR concluído para: {os.path.basename(pdf_path)}")
        return texto_total.strip()
    except Exception as e:
        print(f"  -> Falha GERAL no OCR para '{os.path.basename(pdf_path)}': {e}")
        return ""

# Função inteligente que tenta PyPDF2 e usa OCR como fallback
def extrair_texto_inteligente(pdf_path, limiar_ocr=150):
    """
    Tenta extrair texto com PyPDF2. Se o texto for muito curto (abaixo do limiar),
    tenta usar OCR. Retorna o texto mais longo obtido.
    """
    print(f"Processando: {os.path.basename(pdf_path)}")
    texto_normal = ""
    texto_ocr = ""

    texto_normal = extrair_texto_pypdf2(pdf_path)

    if len(texto_normal) < limiar_ocr:
        print(f"  -> Texto PyPDF2 curto ({len(texto_normal)} chars). Ativando OCR...")
        texto_ocr = extrair_texto_ocr(pdf_path)

        if len(texto_ocr) > len(texto_normal):
             print(f"  -> Usando resultado do OCR ({len(texto_ocr)} chars).")
             return texto_ocr
        else:
             print(f"  -> Resultado do OCR não foi melhor ({len(texto_ocr)} chars). Mantendo resultado PyPDF2.")
             return texto_normal
    else:
        # print(f"  -> Extraído via PyPDF2 ({len(texto_normal)} chars).")
        return texto_normal

print("Funções auxiliares de extração definidas.")

Funções auxiliares de extração definidas.


In [20]:
# === CÉLULA 5: Carregamento dos Documentos ===

documentos_carregados = [] # Lista para guardar os Documentos LangChain

print("\n--- Iniciando Carregamento de Documentos ---")
# Verifica se o CAMINHO_PASTA_PDFS realmente existe e não está vazio antes de prosseguir
if not os.path.exists(CAMINHO_PASTA_PDFS) or not os.listdir(CAMINHO_PASTA_PDFS):
     print(f"🚨 Erro Crítico: Pasta de PDFs '{CAMINHO_PASTA_PDFS}' não encontrada ou vazia. Verifique a Célula 3 e o repositório.")
     # Define a lista como vazia para as próximas células saberem que falhou
     documentos_carregados = []
else:
    # Lista os arquivos PDF encontrados para processamento
    arquivos_pdf_encontrados = [f for f in os.listdir(CAMINHO_PASTA_PDFS) if f.lower().endswith('.pdf')]
    print(f"Encontrados {len(arquivos_pdf_encontrados)} arquivo(s) PDF em '{CAMINHO_PASTA_PDFS}'.")

    # Tenta carregar usando a extração inteligente para cada arquivo

    for nome_arquivo in arquivos_pdf_encontrados:
        caminho_completo = os.path.join(CAMINHO_PASTA_PDFS, nome_arquivo)
        texto_extraido = extrair_texto_inteligente(caminho_completo, limiar_ocr=150)
        if texto_extraido:
            doc = Document(page_content=texto_extraido, metadata={'source': caminho_completo})
            documentos_carregados.append(doc)
            print(f"  -> OK: {nome_arquivo} ({len(texto_extraido)} chars)") # Confirmação
        else:
             print(f"  -> Falha/Vazio: {nome_arquivo}")


    print(f"\n✅ Carregamento concluído: {len(documentos_carregados)} documentos processados e carregados.")

# Verificação final
if documentos_carregados:
    print(f"\n--- Processamento Finalizado ---")
    print(f"Total de Documentos LangChain prontos para divisão: {len(documentos_carregados)}")
else:
    print("\n🚨 ATENÇÃO: Nenhum documento foi carregado com sucesso. Verifique os logs acima.")


--- Iniciando Carregamento de Documentos ---
Encontrados 9 arquivo(s) PDF em '/content/MBACDIA-IAGen/arquivos-projeto'.
Processando: compress ocr mou-anpd-aepd-pt_compressed.pdf
  -> OK: compress ocr mou-anpd-aepd-pt_compressed.pdf (14473 chars)
Processando: compress ocr cade act-tarjado-compactado_compressed.pdf
  -> OK: compress ocr cade act-tarjado-compactado_compressed.pdf (35445 chars)
Processando: compress ocr CGE MG SUPER_PR - 4556235 - Contrato (1)_compressed.pdf
  -> OK: compress ocr CGE MG SUPER_PR - 4556235 - Contrato (1)_compressed.pdf (32508 chars)
Processando: compress ocr TSE-acordo-cooperacao-tecnica-anpd-lgpd_compressed.pdf
  -> OK: compress ocr TSE-acordo-cooperacao-tecnica-anpd-lgpd_compressed.pdf (27441 chars)
Processando: compress ocr CGU Acordo_de_cooperacao_ANPD (1)_compressed.pdf
  -> OK: compress ocr CGU Acordo_de_cooperacao_ANPD (1)_compressed.pdf (36755 chars)
Processando: compress ocr EXTRATO_CARTA_DE_ACEITE_DOU_compressed.pdf
  -> OK: compress ocr EXTRATO_

In [21]:
# === CÉLULA 6: Divisão dos Documentos em Chunks ===

chunks = [] # Inicializa a lista de chunks

if documentos_carregados:
    print("\n--- Iniciando Divisão em Chunks ---")
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,        # Tamanho do chunk
        chunk_overlap=150,      # Sobreposição
        length_function=len,
        separators=["\n\n", "\n", " ", ""], # Separadores comuns
        keep_separator=False,
        add_start_index=True,   # Adiciona metadado com índice inicial
    )

    chunks = text_splitter.split_documents(documentos_carregados)
    print(f"✅ Documentos divididos em {len(chunks)} chunks.")


    tamanhos = [len(c.page_content) for c in chunks]
    print(f"   Tamanhos (Min/Médio/Max): {min(tamanhos)} / {int(np.mean(tamanhos))} / {max(tamanhos)}")

else:
    print("\n⚠️ Não há documentos carregados para dividir. Pule esta célula ou corrija a Célula 5.")


--- Iniciando Divisão em Chunks ---
✅ Documentos divididos em 256 chunks.
   Tamanhos (Min/Médio/Max): 135 / 945 / 1000


In [22]:
# === CÉLULA 7: Criação do Banco de Vetores (FAISS) ===

banco_vetores = None # Inicializa a variável

if chunks: # Só prossiga se houver chunks
    print("\n--- Iniciando Criação do Banco de Vetores FAISS ---")
    try:
        embeddings = OpenAIEmbeddings() # Modelo padrão: text-embedding-ada-002

        print(f"Gerando embeddings e criando índice FAISS para {len(chunks)} chunks...")
        banco_vetores = FAISS.from_documents( # Usa os chunks
            documents=chunks,
            embedding=embeddings
        )
        print(f"✅ Banco de vetores FAISS criado com sucesso com {banco_vetores.index.ntotal} vetores.")

        # Opcional: Salvar o índice localmente
        # try:
        #    indice_path = "meu_indice_faiss_colab"
        #    banco_vetores.save_local(indice_path)
        #    print(f"✅ Índice FAISS salvo localmente em '{indice_path}'.")
        # except Exception as save_err:
        #    print(f"⚠️ Erro ao salvar índice FAISS: {save_err}")

    except Exception as e:
        print(f"❌ Erro ao criar o banco de vetores FAISS: {e}")
        traceback.print_exc()
else:
    print("\n⚠️ Não há chunks para criar o banco de vetores. Pule esta célula ou corrija a Célula 6.")


--- Iniciando Criação do Banco de Vetores FAISS ---
Gerando embeddings e criando índice FAISS para 256 chunks...
✅ Banco de vetores FAISS criado com sucesso com 256 vetores.


In [23]:
# === CÉLULA 7.5: Salvar Índice FAISS ===
if banco_vetores:
    NOME_PASTA_INDICE = "faiss_index_anpd_acts"
    try:
        banco_vetores.save_local(NOME_PASTA_INDICE)
        print(f"✅ Índice FAISS salvo com sucesso na pasta '{NOME_PASTA_INDICE}'.")
        print("   Arquivos criados:", os.listdir(NOME_PASTA_INDICE))
    except Exception as e:
        print(f"❌ Erro ao salvar índice FAISS localmente: {e}")
else:
    print("⚠️ Banco de vetores não existe, não é possível salvar o índice.")

✅ Índice FAISS salvo com sucesso na pasta 'faiss_index_anpd_acts'.
   Arquivos criados: ['index.faiss', 'index.pkl']


In [24]:
# === CÉLULA 8: Configuração e Teste da Chain RetrievalQA ===

qa_chain_instance = None # Inicializa a variável

if banco_vetores: # Só prossiga se o banco de vetores existir
    print("\n--- Configurando a Chain RetrievalQA ---")
    try:
        # Configurar o Retriever
        retriever = banco_vetores.as_retriever(
            search_type="similarity", # Busca por similaridade
            search_kwargs={'k': 6}    # Retorna os 6 chunks mais relevantes
        )
        print(f"Retriever configurado (k={retriever.search_kwargs.get('k', 'Padrão')}).")

        # Configurar o LLM
        llm = ChatOpenAI(
            model_name='gpt-3.5-turbo',
            temperature=0.3
        )
        print(f"LLM configurado: {llm.model_name} (Temperature={llm.temperature})")

        # Criar a Chain RetrievalQA
        qa_chain_instance = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff", # Combina contexto e pergunta num único prompt
            retriever=retriever,
            return_source_documents=True, # Retorna os chunks usados
            chain_type_kwargs={"prompt": None} # Usar prompt padrão do Langchain para "stuff"
        )
        print("✅ Chain RetrievalQA criada com sucesso!")

        # --- Teste ---
        print("\n--- Testando a Chain ---")
        # Use uma pergunta relevante para os seus documentos ACTs
        pergunta_teste = "Qual o objeto do acordo de cooperação técnica entre ANPD e TSE?"
        # pergunta_teste = "Quais as obrigações da ANPD no acordo com o CADE?"
        # pergunta_teste = "Qual a vigência do acordo entre CGU e ANPD?"

        print(f"\nPergunta: {pergunta_teste}")

        try:
             # Use .invoke para Langchain >= 0.1.0
             resposta = qa_chain_instance.invoke({"query": pergunta_teste})
             # Para versões mais antigas, pode ser necessário usar:
             # resposta = qa_chain_instance({"query": pergunta_teste})

             print("\n--- Resposta Gerada ---")
             print(resposta.get('result', 'Nenhuma resposta encontrada.'))

             print("\n--- Documentos Fonte Utilizados ---")
             if resposta.get('source_documents'):
                 fontes_usadas = set() # Para evitar listar a mesma fonte múltiplas vezes
                 for i, doc in enumerate(resposta['source_documents']):
                     source_file = os.path.basename(doc.metadata.get('source', 'N/A'))
                     if source_file not in fontes_usadas:
                          print(f"- {source_file}")
                          fontes_usadas.add(source_file)
                     # Opcional: mostrar trecho do chunk
                     # print(f"  Chunk (Início: {doc.metadata.get('start_index', '?')}): {doc.page_content[:150]}...")
                 if not fontes_usadas:
                      print("Nenhum nome de arquivo encontrado nos metadados dos documentos fonte.")
             else:
                 print("Nenhum documento fonte retornado pela chain.")

        except Exception as query_err:
             print(f"\n❌ Erro ao executar a pergunta na chain: {query_err}")
             traceback.print_exc()


    except Exception as e:
        print(f"❌ Erro ao configurar a chain QA: {e}")
        traceback.print_exc()

else:
    print("\n⚠️ Banco de vetores não foi criado. Não é possível configurar ou testar a Chain QA.")


print("\n--- CONTEÚDO dos Documentos Fonte Recuperados ---")
if resposta.get('source_documents'):
    for i, doc in enumerate(resposta['source_documents']):
        print(f"--- Fonte {i+1} ({os.path.basename(doc.metadata.get('source', 'N/A'))}) ---")
        print(doc.page_content) # Imprime o conteúdo completo do chunk
        print("-" * 20)
else:
    print("Nenhum documento fonte recuperado.")


--- Configurando a Chain RetrievalQA ---
Retriever configurado (k=6).
LLM configurado: gpt-3.5-turbo (Temperature=0.3)
✅ Chain RetrievalQA criada com sucesso!

--- Testando a Chain ---

Pergunta: Qual o objeto do acordo de cooperação técnica entre ANPD e TSE?

--- Resposta Gerada ---
O objeto do acordo de cooperação técnica entre a Autoridade Nacional de Proteção de Dados (ANPD) e o Tribunal Superior Eleitoral (TSE) é a realização de ações conjuntas para orientação e monitoramento da implementação da Lei Geral de Proteção de Dados (LGPD) no contexto eleitoral.

--- Documentos Fonte Utilizados ---
- compress ocr TSE-acordo-cooperacao-tecnica-anpd-lgpd_compressed.pdf
- compress ocr CGE MG SUPER_PR - 4556235 - Contrato (1)_compressed.pdf
- compress ocr act-senacon_ocultado (1)_compressed.pdf
- compress ocr cade act-tarjado-compactado_compressed.pdf

--- CONTEÚDO dos Documentos Fonte Recuperados ---
--- Fonte 1 (compress ocr TSE-acordo-cooperacao-tecnica-anpd-lgpd_compressed.pdf) ---
conc

In [25]:
# ========================================================
# === CÉLULA 9: Criar Arquivo da Aplicação (app.py) ===
# ========================================================

FAISS_INDEX_PATH_APP = "faiss_index_anpd_acts"

OPENAI_SECRET_NAME_APP = "OPENAI_API_KEY"

# Ajuste os parâmetros do LLM e Retriever para o App Streamlit.
K_RETRIEVER_APP = 4
LLM_MODEL_APP = 'gpt-4o-mini'
LLM_TEMP_APP = 0.3

# Personalize os textos da interface do usuário.
APP_TITLE = "🖊📑 Chatbot Consulta ACTs ANPD"
APP_HEADER = "Consulte informações sobre Acordos de Cooperação Técnica da ANPD"
APP_INPUT_LABEL = "Digite sua pergunta sobre os ACTs:"
APP_BUTTON_TEXT = "Buscar Resposta"


# Cria o arquivo app.py com o código do Streamlit
print(f"\n--- Criando arquivo app.py ---")
# Usamos uma string multi-linha normal e formatamos as variáveis do NOTEBOOK nela.
# As f-strings que devem ser interpretadas pelo app.py em runtime precisam ter
# suas chaves escapadas com chaves duplas {{ variavel }}.
writefile_content = f"""
import streamlit as st
import os
import time # Para simular processamento
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
import traceback # Para mostrar erros no app

# --- Configurações e Carregamento de Recursos (com cache) ---

# Carregar Chave API (Prioriza st.secrets)
OPENAI_API_KEY = None
try:
    # Tenta carregar do secrets.toml (para deploy no Streamlit Cloud)
    OPENAI_API_KEY = st.secrets["{OPENAI_SECRET_NAME_APP}"]
    print("OpenAI Key carregada do st.secrets") # Log interno
except (FileNotFoundError, KeyError):
    print("Secret '{OPENAI_SECRET_NAME_APP}' não encontrado no st.secrets. Tentando variável de ambiente...")
    # Tenta carregar de variáveis de ambiente
    OPENAI_API_KEY = os.environ.get('{OPENAI_SECRET_NAME_APP}')
    if OPENAI_API_KEY:
        print("OpenAI Key carregada da variável de ambiente.")
    else:
        st.error("Chave API da OpenAI ('{OPENAI_SECRET_NAME_APP}') não configurada! Configure em st.secrets ou variável de ambiente.")
        st.stop() # Para a execução se não tiver a chave

# Define a variável de ambiente para Langchain usar
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# Caminho para o índice FAISS salvo
FAISS_INDEX_PATH = "{FAISS_INDEX_PATH_APP}"

@st.cache_resource(show_spinner="Carregando base de conhecimento (índice FAISS)...")
def load_faiss_index(index_path):
    if not os.path.exists(index_path):
         # CORREÇÃO: Escapar chaves ao redor de index_path
         st.error(f"Pasta do índice FAISS não encontrada em '{{index_path}}'. Verifique se a pasta existe no repositório junto com app.py.")
         st.stop()
    try:
        embeddings = OpenAIEmbeddings()
        vector_store = FAISS.load_local(
            index_path,
            embeddings,
            allow_dangerous_deserialization=True
        )
        return vector_store
    except Exception as e:
        # CORREÇÃO: Escapar chaves ao redor de e
        st.error(f"Erro ao carregar índice FAISS: {{e}}")
        st.error(traceback.format_exc())
        st.stop()

@st.cache_resource(show_spinner="Preparando o assistente (chain QA)...")
def create_qa_chain(_vector_store):
    try:
        # Usa variáveis definidas no início do script app.py (não precisa escapar)
        llm = ChatOpenAI(model_name='{LLM_MODEL_APP}', temperature={LLM_TEMP_APP})
        retriever = _vector_store.as_retriever(search_kwargs={{'k': {K_RETRIEVER_APP}}})
        qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=retriever,
            return_source_documents=True
        )
        return qa_chain
    except Exception as e:
         # CORREÇÃO: Escapar chaves ao redor de e
         st.error(f"Erro ao criar a chain QA: {{e}}")
         st.error(traceback.format_exc())
         st.stop()

# --- Carregar Recursos ---
vector_store_app = load_faiss_index(FAISS_INDEX_PATH)
qa_chain_app = create_qa_chain(vector_store_app)
st.success("Assistente pronto!") # Mensagem de sucesso após carregar tudo

# --- Interface do Usuário Streamlit ---
st.title("{APP_TITLE}")
st.markdown("{APP_HEADER}")
st.divider()

with st.form("input_form"):
    user_question = st.text_area("{APP_INPUT_LABEL}", key="user_input", height=100)
    submitted = st.form_submit_button("{APP_BUTTON_TEXT}")

    if submitted:
        if user_question:
            with st.spinner("Buscando informações nos documentos..."):
                try:
                    start_time = time.time()
                    resposta = qa_chain_app.invoke({{"query": user_question}})
                    end_time = time.time()

                    st.markdown("### Resposta:")
                    st.info(resposta.get('result', "Não foi possível obter uma resposta."))
                    st.caption(f"Tempo de resposta: {{end_time - start_time:.2f}} segundos") # Escapar aqui também

                    with st.expander("Ver Documentos Fonte Utilizados"):
                        if resposta.get('source_documents'):
                            fontes_usadas = set()
                            for doc in resposta['source_documents']:
                                source_file = os.path.basename(doc.metadata.get('source', 'N/A'))
                                if source_file not in fontes_usadas:
                                    # Usar st.markdown para formatar como código
                                    st.markdown(f"- `{source_file}`")
                                    fontes_usadas.add(source_file)
                            if not fontes_usadas:
                                 st.write("Nenhuma fonte específica identificada nos metadados.")
                        else:
                            st.write("Nenhum documento fonte foi retornado pela chain.")

                except Exception as e:
                    # CORREÇÃO: Escapar chaves ao redor de e
                    st.error(f"Ocorreu um erro ao processar sua pergunta:")
                    st.exception(e) # Mostra o erro detalhado no app
        else:
            st.warning("Por favor, digite uma pergunta.")

st.divider()
st.caption("MBA IA & Big Data - Projeto GenAI RAG - Camila Falchetto")

"""

# Salva o conteúdo no arquivo app.py
try:
  with open("app.py", "w") as f:
    f.write(writefile_content)
  print("✅ Arquivo app.py criado/sobrescrito com sucesso.")
except Exception as e:
  print(f"❌ Erro ao escrever o arquivo app.py: {e}")


--- Criando arquivo app.py ---
✅ Arquivo app.py criado/sobrescrito com sucesso.


In [26]:
# ===================================================================
# === CÉLULA 10: Iniciar Streamlit em Background e Expor com Ngrok ===
# ===================================================================
import os
import time
from pyngrok import ngrok
from google.colab import userdata

print("--- Iniciando Streamlit em Background ---")

# Comando para rodar streamlit em background e salvar logs
# nohup: evita que o processo morra se a conexão 'cair'
# > streamlit_log.txt: redireciona a saída padrão para o arquivo
# 2>&1: redireciona a saída de erro para o mesmo arquivo da saída padrão
# &: executa em background
streamlit_command = "nohup streamlit run app.py --server.port 8501 --server.enableCORS false --server.enableXsrfProtection false > streamlit_log.txt 2>&1 &"

# Executa o comando para iniciar o Streamlit
print(f"Executando: {streamlit_command}")
os.system(streamlit_command)
print("Comando para iniciar Streamlit enviado para background.")
print("Aguardando alguns segundos para o servidor Streamlit iniciar...")
time.sleep(10) # Pausa por 10 segundos (ajuste se necessário)

# Verifica se o Streamlit parece estar rodando (opcional, checando log)
print("\nVerificando log do Streamlit (streamlit_log.txt)...")
!tail streamlit_log.txt # Mostra as últimas linhas do log

print("\n--- Configurando Ngrok ---")

NGROK_SECRET_NAME = 'NGROK_AUTHTOKEN'

try:
    ngrok_auth = userdata.get(NGROK_SECRET_NAME)
    if not ngrok_auth:
         raise ValueError(f"Secret '{NGROK_SECRET_NAME}' não encontrado ou vazio.")

    # Mata processos ngrok anteriores se existirem
    print("Tentando limpar processos ngrok antigos...")
    ngrok.kill()

    # Configura e conecta o ngrok
    ngrok.set_auth_token(ngrok_auth)
    print("Token Ngrok configurado.")
    # Expor a porta 8501
    public_url = ngrok.connect(8501, proto='http')
    print("\n=======================================================================")
    print(f"✅ Aplicação Streamlit deve estar rodando em background.")
    print(f"   Acesso público temporário via: {public_url}")
    print("   (Este túnel permanecerá ativo enquanto esta célula Colab estiver ativa)")
    print("   Para VER os logs do Streamlit, execute: !cat streamlit_log.txt")
    print("   Para PARAR TUDO: Interrompa/Reinicie o ambiente de execução do Colab.")
    print("=======================================================================")

except Exception as e:
    print(f"\n❌ Erro ao configurar ou iniciar ngrok: {e}")
    print(f"   Verifique se o token '{NGROK_SECRET_NAME}' está nos secrets do Colab.")
    print(f"   Verifique também o log do Streamlit ('streamlit_log.txt') para erros.")
    traceback.print_exc()


--- Iniciando Streamlit em Background ---
Executando: nohup streamlit run app.py --server.port 8501 --server.enableCORS false --server.enableXsrfProtection false > streamlit_log.txt 2>&1 &
Comando para iniciar Streamlit enviado para background.
Aguardando alguns segundos para o servidor Streamlit iniciar...

Verificando log do Streamlit (streamlit_log.txt)...



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.72.62.226:8501


--- Configurando Ngrok ---
Tentando limpar processos ngrok antigos...
Token Ngrok configurado.

✅ Aplicação Streamlit deve estar rodando em background.
   Acesso público temporário via: NgrokTunnel: "https://9786-34-72-62-226.ngrok-free.app" -> "http://localhost:8501"
   (Este túnel permanecerá ativo enquanto esta célula Colab estiver ativa)
   Para VER os logs do Streamlit, execute: !cat streamlit_log.txt
   Para PARAR TUDO: Interrompa/Reinicie o ambiente d